<a href="https://colab.research.google.com/github/mars-ef/Trustpilot_Scraper/blob/main/Trustpilot_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Analytics with Adam video on how to use https://www.youtube.com/watch?v=VD0wIiLVSRA

from time import sleep
import requests
from google.colab import files
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text())

brand = [] #Same as company
users = []
ratings = []
locations = []
experiences = [] #Date of experience being reviewed
dates = [] #Date review posted or updated
reviews = [] #Content of review
replies = [] #Whether business replied
reply_dates = [] #Date business replied
from_page = 1
to_page = 5
company = 'www.revolut.com' #Take the end bit of the trustpilot URL. E.g. for https://www.trustpilot.com/review/wise.com you'd take 'wise.com'

for i in range(from_page, to_page + 1):
    result = requests.get(f"https://www.trustpilot.com/review/{company}?page={i}")
    soup = BeautifulSoup(result.content, 'html.parser')

    soup2list(soup.find_all('span', {'class', 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}), users)
    soup2list(soup.find_all('div', {'class', 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}), locations)
    soup2list(soup.find_all('div', {'class', 'styles_reviewHeader__iU9Px'}), dates)
    soup2list(soup.find_all('div', {'class', 'styles_reviewHeader__iU9Px'}), ratings, attr='data-service-review-rating')
    soup2list(soup.find_all('div', {'class', 'styles_reviewContent__0Q2Tg'}), reviews)

    soup2list(soup.find_all('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17', 'data-service-review-date-of-experience-typography': 'true'}), experiences)
    replies_ = soup.find_all('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17 typography_weight-heavy__E1LTj styles_replyCompany__ro_yX'}) # Changed variable name to replies_
    dates_ = soup.find_all('time', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_replyDate__Iem0_'})

    # Makes sure records added even if no business reply
    for review in soup.find_all('div', {'class', 'styles_reviewContent__0Q2Tg'}):
        if len(replies_) > 0:
            replies.append(1)
            reply_dates.append(dates_.pop(0).get_text())
            replies_.pop(0)
        else:
            replies.append(0)
            reply_dates.append('')

   # To avoid throttling
    sleep(1)

# Remove extra words from various columns before adding to DataFrame
def remove_extra_words(text, words_to_remove):
    for word in words_to_remove:
        text = text.replace(word, '').strip()
    return text

dates = [remove_extra_words(date, ['Updated', 'Verified', 'Redirected']) for date in dates]
content = [remove_extra_words(content, ['Date of experience:.*']) for content in reviews]
experiences = [remove_extra_words(experience, ['Date of experience:', 'Updated', 'Verified', 'Redirected']) for experience in experiences]
reply_dates = [remove_extra_words(reply_date, ['Updated', 'Verified', 'Redirected']) for reply_date in reply_dates]

# Create the DataFrame
review_data = pd.DataFrame({
    'brand': company,
    'username': users,
    'location': locations,
    'date': dates,
    'content': content,
    'rating': ratings,
    'date_of_experience': experiences,
    'reply_from_business': replies,
    'reply_date': reply_dates
})

# Removes 'invited' from 'date' and adds the invited column
review_data['invited'] = 0
for index, row in review_data.iterrows():
    if 'Invited' in row['date']:
        review_data.loc[index, 'invited'] = 1
        review_data.loc[index, 'date'] = row['date'].replace('Invited', '').strip()

#Converts the data in 'date' and 'reply_date' to Yyyy-Mm-Dd format
def convert_relative_date(date_str):
  if date_str == 'Just now':
    return datetime.now().strftime('%Y-%m-%d')
  elif date_str == 'A day ago':
    return (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
  elif 'days ago' in date_str:
    days = int(date_str.split(' ')[0])
    return (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
  elif 'hours ago' in date_str:
    hours = int(date_str.split(' ')[0])
    return (datetime.now() - timedelta(hours=hours)).strftime('%Y-%m-%d')
  elif 'minutes ago' in date_str:
    minutes = int(date_str.split(' ')[0])
    return (datetime.now() - timedelta(minutes=minutes)).strftime('%Y-%m-%d')
  else:
    try:
      return datetime.strptime(date_str, '%B %d, %Y').strftime('%Y-%m-%d')
    except ValueError:
      return date_str

review_data['date'] = review_data['date'].apply(convert_relative_date)
review_data['date_of_experience'] = review_data['date_of_experience'].apply(convert_relative_date)
review_data['reply_date'] = review_data['reply_date'].apply(convert_relative_date)

In [ ]:
review_data

,brand,username,location,date,content,rating,date_of_experience,reply_from_business,reply_date,invited
0,www.revolut.com,Moeno Mokati,FI,2024-09-24,"When issues pop upWhen issues pop up, it’s rea...",5,2024-09-24,1,2024-09-26,1
1,www.revolut.com,Uma Purohit,GB,2024-09-24,One card does it allWhilst travelling I find i...,5,2024-09-24,1,2024-09-28,1
2,www.revolut.com,Nnabiko Augustine Chime,GB,2024-09-24,I'm extremely satisfied with RevolutI'm extrem...,5,2024-09-24,0,,1
3,www.revolut.com,Dave D,GB,2024-09-25,An example to the rest in how to do it We have...,5,2024-09-25,0,,0
4,www.revolut.com,Mārtiņš Kronbergs,LV,2024-09-24,"For Ultra planFor Ultra plan, better to have u...",4,2024-09-24,0,,1
...,...,...,...,...,...,...,...,...,...,...
95,www.revolut.com,Fero Turna,SK,2024-09-24,Much better than banks in my countryMuch bette...,5,2024-09-24,0,,0
96,www.revolut.com,Stephen Taylor,GB,2024-09-23,if could give 0 stars I wouldif could give 0 s...,1,2024-09-23,0,,0
97,www.revolut.com,Francesca Harrall,GB,2024-09-26,Deactivated account and no reason given by the...,1,2024-09-26,0,,0
98,www.revolut.com,sarah davies,GB,2024-09-23,All is well until something goes wrong…All is ...,1,2024-09-23,0,,0


In [ ]:
# Convert the DataFrame to a CSV file and download it
review_data.to_csv('reviews.csv', index=False)
files.download('reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>